TF-IDF

In [76]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [77]:
# Charger le fichier de mots vides (stopwords) personnalisés
french_stopwords_path = '../../app/models/french.txt'

# Vérifier si le fichier existe et le lire
if os.path.exists(french_stopwords_path):
    with open(french_stopwords_path, 'r', encoding='utf-8') as file:
        french_stopwords = file.read().splitlines()  # Lire chaque ligne comme un mot séparé
    print(french_stopwords[:10])  # 10 premiers mots 
else:
    print(f"Le fichier french.txt est introuvable dans {french_stopwords_path}")

['a', 'ai', 'aie', 'aient', 'aies', 'ait', 'alors', 'as', 'au', 'aucun']


In [78]:
# Charger les données depuis le CSV
df = pd.read_csv('../../RGBD/table_produits/produits.csv')

In [79]:
# les descriptions des produits
descriptions = df['description'].tolist()

In [80]:
# Initialisation du vectoriseur TF-IDF avec la liste personnalisée de mots vides
vectorizer = TfidfVectorizer(stop_words=french_stopwords)

In [81]:
# Transformation des descriptions en vecteurs TF-IDF
tfidf_matrix = vectorizer.fit_transform(descriptions)

In [82]:
# Fonction pour recommander des produits basés sur une requête utilisateur
def recommend_by_query(query, tfidf_matrix, vectorizer, top_n=5):
    # Transformation de la requête en vecteur TF-IDF
    query_tfidf = vectorizer.transform([query])
    
    # Calcul de la similarité cosinus entre la requête et les descriptions des produits
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    
    # Trier les indices des produits les plus similaires
    similar_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Afficher les produits recommandés
    print(f"Produits recommandés pour la requête '{query}':\n")
    for idx in similar_indices:
        print(f"Produit {idx+1}: {df['description'][idx]} - Similarité: {cosine_similarities[idx]:.4f}")


In [83]:
# Exemple d'appel avec la requête "Saveur menthe"
recommend_by_query("Saveur menthe", tfidf_matrix, vectorizer)

Produits recommandés pour la requête 'Saveur menthe':

Produit 1274: Le Menthe Verte Glacée reprend lessaveurs fraîches d'une menthe chlorophylle, dans une recette signée PULP. Prêt à vaper(PAV), il vous est proposé dans plusieurs taux de nicotine au choix (0, 3, 6 et 12mg/ml), à sélectionner à la commande. Le Menthe Verte Glacée est élaboré à partir d'une base PG/VG de 70/30, et vous est livré dans une fiole de 10ml. Le Menthe Verte Glacée est un e-liquide exclusivement destiné au réservoir de votre cigarette électronique. Il est fabriqué en France par Pulp, à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale. Le e-liquide Menthe Verte Glacée vous est livré dans un flacon de 10ml, équipé d'un bouchon avec sécurité enfant et d'un embout fin facilitant le remplissage de votre réservoir. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque PULP conçoivent leurs multiples recettes, et ce, toujours

In [26]:
# Requête de l'utilisateur
requete_utilisateur = "Saveur menthe"

In [27]:
X = vectorizer.fit_transform(df['description'])

In [28]:
# Transformer la requête de l'utilisateur en vecteur TF-IDF
requete_vector = vectorizer.transform([requete_utilisateur])

In [29]:
# Calculer la similarité cosinus entre la requête et les descriptions des produits
similarites = cosine_similarity(requete_vector, X)

In [30]:
# Afficher les produits les plus similaires (triés par similarité décroissante)
df['similarity'] = similarites.flatten()
df_sorted = df.sort_values(by='similarity', ascending=False)

In [32]:
df.columns

Index(['url', 'nom_produit', 'img_produit', 'prix_produit', 'contenance',
       'pg_vg', 'origine', 'frais', 'surbooste', 'saveur', 'description',
       'brand', 'gout', 'info_brand', 'id_produit', 'similarity'],
      dtype='object')

In [33]:

print(df_sorted[['id_produit', 'description', 'similarity']])

      id_produit                                        description  \
1273        1275  Le Menthe Verte Glacée reprend lessaveurs fraî...   
1342        1344  Pulp vous propose de vapoter une saveur de boi...   
1249        1251  Fabriqué en France par Alfaliquid, le Menthe P...   
99           100  Si les Français de VDLV n'ont plus besoin d'êt...   
1263        1265  Le Menthe Glaciale est un e-liquide à booster ...   
...          ...                                                ...   
701          703  Le Cassis Noir est un e-liquide à booster au f...   
699          701  Les vendanges ont été fructueuses chez The Fuu...   
695          697  Le Fraise Kiwi Salt est un e-liquide avec nico...   
685          687  Le Fraise Grenade est un e-liquide fabriqué en...   
1530        1533  Voici la recette du Blond Sec de chez Alfaliqu...   

      similarity  
1273    0.346166  
1342    0.326076  
1249    0.310499  
99      0.306293  
1263    0.275879  
...          ...  
701     0.0000

 scores de similarité entre une requête donnée (par exemple, "Saveur menthe") et les descriptions de produits dans votre base de données. Voici ce que cela signifie et comment l'utiliser :
 Ces scores sont calculés en utilisant une mesure de similarité (la cosine similarity) qui permet de quantifier à quel point une description de produit correspond à une requête (comme "Saveur menthe"). Plus le score est élevé, plus la description du produit est similaire à la requête. Par exemple, un score de 0.346166 est plus proche d'un score parfait de 1 que d'un score de 0, ce qui signifie que le produit associé est plus pertinent pour la requête. Un score de 0.000000 indique une absence de similarité ou une correspondance faible avec la requête.  En triant les produits par ordre décroissant de similarité, vous obtiendrez les produits les plus pertinents pour la requête en haut de la liste.

In [36]:
# Recommander les 3 produits les plus similaires
produits_recommandes = df_sorted[['id_produit', 'description', 'gout']].head(3)
print("\nProduits recommandés :")
print(produits_recommandes)



Produits recommandés :
      id_produit                                        description    gout
1273        1275  Le Menthe Verte Glacée reprend lessaveurs fraî...  menthe
1342        1344  Pulp vous propose de vapoter une saveur de boi...   frais
1249        1251  Fabriqué en France par Alfaliquid, le Menthe P...  menthe


benchmark
Le benchmark concerne essentiellement la performance et la précision du modèle de recommandation basé sur la similarité cosinus et les vecteurs TF-IDF.

1. Temps de traitement (Latence)
Objectif : Mesurer le temps nécessaire pour traiter une requête utilisateur, c'est-à-dire, combien de temps prend votre système pour transformer la requête en vecteur TF-IDF, calculer la similarité cosinus et retourner les produits recommandés.
Comment mesurer : Vous pouvez utiliser la fonction time.time() pour mesurer le temps d'exécution entre le début et la fin du processus.

In [84]:
import time

start_time = time.time()
recommend_by_query("Saveur menthe", tfidf_matrix, vectorizer)
end_time = time.time()
print(f"Temps de traitement : {end_time - start_time:.4f} secondes")


Produits recommandés pour la requête 'Saveur menthe':

Produit 1274: Le Menthe Verte Glacée reprend lessaveurs fraîches d'une menthe chlorophylle, dans une recette signée PULP. Prêt à vaper(PAV), il vous est proposé dans plusieurs taux de nicotine au choix (0, 3, 6 et 12mg/ml), à sélectionner à la commande. Le Menthe Verte Glacée est élaboré à partir d'une base PG/VG de 70/30, et vous est livré dans une fiole de 10ml. Le Menthe Verte Glacée est un e-liquide exclusivement destiné au réservoir de votre cigarette électronique. Il est fabriqué en France par Pulp, à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale. Le e-liquide Menthe Verte Glacée vous est livré dans un flacon de 10ml, équipé d'un bouchon avec sécurité enfant et d'un embout fin facilitant le remplissage de votre réservoir. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque PULP conçoivent leurs multiples recettes, et ce, toujours

In [108]:
import time

start_time = time.time()
recommend_by_query("Propose moi un produit avec lime", tfidf_matrix, vectorizer)
end_time = time.time()
print(f"Temps de traitement : {end_time - start_time:.4f} secondes")

Produits recommandés pour la requête 'Propose moi un produit avec lime':

Produit 277: Le Bloody Lime est un e liquide au formatprêt à l'emploipour cigarette électronique. Il rejoint la célèbre gamme Fruizee, fabriquée en France par Eliquid France, et vous propose des saveurs de fruits rouges et citron vert. Composé d'arômes alimentaires, de propylène glycol (PG) et de glycérine végétale (VG), le Bloody Lime vous est proposé en50PG/50VG. De même, vous le retrouverez en fiole de type PET de 10ml, munie d'une sécurité enfant et d'un embout fin. Enfin, pour satisfaire tout type de vapoteur, le e-liquide Bloody Lime vous est présenté en différents taux de nicotine au choix, soit 3, 6 et 12mg/ml. Qui est Eliquid France ? Créé en 2013 à Nieul sur Mer en Charente-Maritime, le laboratoire Eliquid France s'est fait une place de choix dans l'univers de la vape grâce à des liquides de qualité, exportant ses créations dans le monde entier. Toujours en recherche d'innovation, la marque française pr

2. Qualité des recommandations (Précision)
Objectif : Évaluer si les produits recommandés sont pertinents par rapport à la requête de l'utilisateur.
Comment mesurer : Vous pouvez tester manuellement la qualité des recommandations en comparant les produits recommandés avec des attentes ou en utilisant des mesures telles que la précision ou le rappel dans un jeu de données annoté.
Précision : Nombre de produits pertinents dans les recommandations divisés par le nombre total de produits recommandés.
Rappel : Nombre de produits pertinents dans les recommandations divisés par le nombre total de produits pertinents.

In [85]:
# Initialisation du vectoriseur TF-IDF avec la liste personnalisée de mots vides
vectorizer = TfidfVectorizer(stop_words=french_stopwords)

# Transformation des descriptions en vecteurs TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['description'])

In [95]:
# Fonction pour recommander des produits basés sur une requête utilisateur et calculer la pertinence
def assign_relevance_to_products(query, tfidf_matrix, vectorizer, threshold=0.5):
    # Transformation de la requête en vecteur TF-IDF
    query_tfidf = vectorizer.transform([query])
    
    # Calcul de la similarité cosinus entre la requête et les descriptions des produits
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()
    
    # Assigner une pertinence en fonction du seuil de similarité
    relevance = [1 if similarity >= threshold else 0 for similarity in cosine_similarities]
    return relevance, cosine_similarities

In [97]:


# Fonction pour afficher les 10 produits les plus pertinents pour une requête
def print_top_10_recommendations(query, threshold=0.5):
    relevance, cosine_similarities = assign_relevance_to_products(query, tfidf_matrix, vectorizer, threshold)
    
    # Créer un DataFrame avec les résultats de similarité et pertinence
    results = pd.DataFrame({
        'description': df['description'],
        'similarity': cosine_similarities,
        'relevance': relevance
    })
    
    # Trier les produits par similarité décroissante
    results_sorted = results.sort_values(by='similarity', ascending=False)
    
    # Afficher les 10 produits les plus pertinents
    top_10 = results_sorted.head(10)
    
    print(f"Les 10 produits les plus pertinents pour la requête : '{query}'\n")
    for idx, row in top_10.iterrows():
        pertinence = 'Pertinent' if row['relevance'] == 1 else 'Non pertinent'
        print(f"Produit {idx+1} - Description : {row['description']} - Similarité : {row['similarity']:.4f} - Pertinence : {pertinence}")

# Exemple d'appel avec la requête "Saveur menthe"
print_top_10_recommendations("Saveur menthe", threshold=0.2)


Les 10 produits les plus pertinents pour la requête : 'Saveur menthe'

Produit 1274 - Description : Le Menthe Verte Glacée reprend lessaveurs fraîches d'une menthe chlorophylle, dans une recette signée PULP. Prêt à vaper(PAV), il vous est proposé dans plusieurs taux de nicotine au choix (0, 3, 6 et 12mg/ml), à sélectionner à la commande. Le Menthe Verte Glacée est élaboré à partir d'une base PG/VG de 70/30, et vous est livré dans une fiole de 10ml. Le Menthe Verte Glacée est un e-liquide exclusivement destiné au réservoir de votre cigarette électronique. Il est fabriqué en France par Pulp, à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale. Le e-liquide Menthe Verte Glacée vous est livré dans un flacon de 10ml, équipé d'un bouchon avec sécurité enfant et d'un embout fin facilitant le remplissage de votre réservoir. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque PULP conçoivent leurs mult

3. Utilisation de mémoire (Consommation mémoire)
Objectif : Mesurer la mémoire utilisée par le modèle et les calculs pour voir si votre solution peut évoluer avec des jeux de données plus volumineux.
Comment mesurer : Utilisez des outils comme psutil ou memory_profiler pour mesurer l'utilisation de la mémoire.

In [98]:
import psutil
process = psutil.Process(os.getpid())
print(f"Utilisation de la mémoire avant recommandation: {process.memory_info().rss / 1024 ** 2:.2f} MB")
recommend_by_query("Saveur menthe", tfidf_matrix, vectorizer)
print(f"Utilisation de la mémoire après recommandation: {process.memory_info().rss / 1024 ** 2:.2f} MB")


Utilisation de la mémoire avant recommandation: 824.96 MB
Produits recommandés pour la requête 'Saveur menthe':

Produit 1274: Le Menthe Verte Glacée reprend lessaveurs fraîches d'une menthe chlorophylle, dans une recette signée PULP. Prêt à vaper(PAV), il vous est proposé dans plusieurs taux de nicotine au choix (0, 3, 6 et 12mg/ml), à sélectionner à la commande. Le Menthe Verte Glacée est élaboré à partir d'une base PG/VG de 70/30, et vous est livré dans une fiole de 10ml. Le Menthe Verte Glacée est un e-liquide exclusivement destiné au réservoir de votre cigarette électronique. Il est fabriqué en France par Pulp, à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale. Le e-liquide Menthe Verte Glacée vous est livré dans un flacon de 10ml, équipé d'un bouchon avec sécurité enfant et d'un embout fin facilitant le remplissage de votre réservoir. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque

 Précision du modèle TF-IDF et amélioration avec d'autres modèles
 Pour améliorer les performances de recommandation, vous pouvez comparer le modèle TF-IDF avec des modèles plus avancés comme BERT, Word2Vec, ou FastText pour obtenir des représentations plus riches des descriptions de produits. Voici un guide pour mesurer la précision du modèle TF-IDF et améliorer les recommandations en utilisant des modèles plus avancés.

In [99]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

c:\Users\antoa\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [100]:
# Charger le modèle BERT et son tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [101]:
# Fonction pour obtenir les embeddings d'une description ou d'une requête
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()  # Moyenne des tokens pour obtenir l'embedding
    return embeddings.numpy()

In [102]:
# Exemple de requête
query = "Saveur menthe"

In [103]:
# Obtenez les embeddings de la requête
query_embedding = get_bert_embeddings(query)

In [104]:
# Charger les descriptions de produits
descriptions = df['description'].tolist()

In [105]:
# Obtenir les embeddings des descriptions
descriptions_embeddings = np.array([get_bert_embeddings(desc) for desc in descriptions])

# Calculer la similarité cosinus entre la requête et les descriptions des produits
similarities = cosine_similarity([query_embedding], descriptions_embeddings).flatten()


In [106]:
# Trier les produits les plus similaires
top_n = 10
similar_indices = similarities.argsort()[-top_n:][::-1]

In [107]:
# Afficher les produits recommandés
for idx in similar_indices:
    print(f"Produit {idx+1}: {df['description'][idx]} - Similarité: {similarities[idx]:.4f}")

Produit 974: Découvrez le e-liquide Strawberry Macarron Salt Nic du fabricant anglais Vape Dinner Lady,un mélange de macaron et de coulis de fraiseà vaper ! Le Strawberry MacaroonSaltNic est un e-liquideprêt à l'emploi aux sels de nicotine. Il se compose d'arômes alimentaires, de propylène glycol et de glycérine végétale dans un ratioPG/VGde 50/50. Disponible en plusieurs 20mg/ml de sels de nicotine, il vous arrivera en fiole PET de 10ml, avec bouchon sécurisé et embout fin propre et pratique. Qui est Vape Dinner Lady ? Vape Dinner Lady est un fabricant anglais de recettes DIY mais aussi de e-liquides aux saveurs complexes comme fruitées et fraîches. Véritable référence dans la vaposphère, soyez certain en choisissant la marque britannique de vapoter un produit abouti dont la qualité n'est plus à prouver Qu'est-ce que les sels de nicotine ? Le sel de nicotine est la forme qui se rapproche le plus de la nicotine présente à l’état naturel dans la plante. En utilisant un e-liquide aux sel

Le TF-IDF Vectorizer est utilisé pour convertir les descriptions des produits et la requête de l'utilisateur en vecteurs.
La similarité cosinus est calculée entre la requête de l'utilisateur et chaque produit pour déterminer les produits les plus proches.
Les résultats sont triés en fonction de la similarité, et les produits les plus similaires à la requête sont affichés.

BM25

In [109]:
# Importer la bibliothèque BM25
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import string

In [110]:
# Fonction de prétraitement (tokenisation et suppression de la ponctuation)
def preprocess(text):
    # Tokenisation et suppression de la ponctuation
    tokens = word_tokenize(text.lower())
    return [token for token in tokens if token not in string.punctuation]

In [111]:
# Appliquer le prétraitement sur toutes les descriptions
preprocessed_descriptions = [preprocess(desc) for desc in df['description']]


In [112]:
# Initialiser le modèle BM25
bm25 = BM25Okapi(preprocessed_descriptions)


In [113]:
# Exemple de requête utilisateur
query = "Saveur cassis lime"


In [114]:
# Prétraiter la requête
preprocessed_query = preprocess(query)

In [49]:
# Calculer la similarité BM25 pour chaque description
scores = bm25.get_scores(preprocessed_query)


In [50]:
# Afficher les scores de similarité pour chaque description
df['similarity_score'] = scores

In [51]:
# Trier les produits par score de similarité
sorted_df = df.sort_values(by='similarity_score', ascending=False)

In [52]:
# Afficher les 5 produits les plus pertinents
print("Produits les plus pertinents pour la requête :")
print(sorted_df[['description', 'similarity_score']].head())

Produits les plus pertinents pour la requête :
                                           description  similarity_score
101  Besoin d'un bon coup de fouet fruité, frais et...         10.749055
369  Le Bloody Lime est un e-liquideà boosterde la ...         10.261856
276  Le Bloody Lime est un e liquide au formatprêt ...         10.119159
28   Le Frozen Lemon And Lime 100ml est un e-liquid...          9.297028
257  Le Lime Berry 100ml est un e-liquide à booster...          9.160395


In [53]:
# Si vous voulez afficher le produit le plus pertinent :
most_relevant_product = sorted_df.iloc[0]
print(f"\nLe produit le plus pertinent est :\n{most_relevant_product['description']} avec un score de similarité de {most_relevant_product['similarity_score']:.4f}")


Le produit le plus pertinent est :
Besoin d'un bon coup de fouet fruité, frais et acidulé ? Essayez donc le e liquide Grapefruit Lime Ice par Frozen Freaks ! Ce revigorantmix d'agrumes et de fraicheurravivera le plaisir de vapoter, bouffée après bouffée. Et, grâce à son généreuxflacon 120ml, vous pourrez le booster à votre convenance ! Un mariage acidulé et frais... Nous vous proposons de vous laisser surprendre par une association mystérieuse et puissante de fruits et d'agrumes. Tout cela s'illustre par un mélange depamplemousseet decitron vertpour la note acidulée. Le tout accompagné par une vague de fraicheur intense pour sublimer la recette. Le Grapefruit Lime Ice est ainsi un e-liquide riche en saveurs et en fraicheur qui vous charmera instantanément ! Une recette puissante que l'on doit au fabricant anglais Frozen Freaks. ...en e-liquide 100ml à booster ! En plus d'être un merveilleux mélange fruité et glacé, le e-liquide Grapefruit Lime Ice par Frozen Freaks propose un maxi for

In [118]:


# Définition de la requête
query = "Saveur cassis lime"
preprocessed_query = preprocess(query)

# Mesure du temps d'exécution
start_time = time.time()
scores = bm25.get_scores(preprocessed_query)
df['similarity_score'] = scores
sorted_df = df.sort_values(by='similarity_score', ascending=False)
end_time = time.time()

# Affichage des résultats et du temps d'exécution
print(f"Temps d'exécution BM25 : {end_time - start_time:.4f} secondes")
print(sorted_df[['description', 'similarity_score']].head())


Temps d'exécution BM25 : 0.0147 secondes
                                           description  similarity_score
101  Besoin d'un bon coup de fouet fruité, frais et...         10.749055
369  Le Bloody Lime est un e-liquideà boosterde la ...         10.261856
276  Le Bloody Lime est un e liquide au formatprêt ...         10.119159
28   Le Frozen Lemon And Lime 100ml est un e-liquid...          9.297028
257  Le Lime Berry 100ml est un e-liquide à booster...          9.160395


evaluations des resultats

In [126]:
def get_tfidf_scores(query, tfidf_matrix, vectorizer):
    query_tfidf = vectorizer.transform([query])  # Transformer la requête en vecteur TF-IDF
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix).flatten()  # Calcul des scores
    return cosine_similarities  # Retourne un tableau de scores


In [127]:
def get_bm25_scores(query, bm25):
    preprocessed_query = preprocess(query)  # Tokenisation de la requête
    scores = bm25.get_scores(preprocessed_query)  # Obtenir les scores BM25
    return scores  # Retourne un tableau de scores


In [129]:
import pandas as pd
import numpy as np

def compare_similarity_scores(query, tfidf_matrix, vectorizer, bm25, preprocessed_descriptions, df, top_n=5):
    # Obtenir les scores de similarité pour TF-IDF et BM25
    tfidf_scores = get_tfidf_scores(query, tfidf_matrix, vectorizer)
    bm25_scores = get_bm25_scores(query, bm25)

    # Ajouter les scores dans le DataFrame
    df['tfidf_score'] = tfidf_scores
    df['bm25_score'] = bm25_scores

    # Trier les résultats par BM25
    sorted_df = df.sort_values(by=['bm25_score', 'tfidf_score'], ascending=[False, False])

    # Afficher les meilleurs résultats
    print(f"\n🔍 Requête : {query}\n")
    print(sorted_df[['nom_produit', 'saveur', 'tfidf_score', 'bm25_score']].head(top_n).to_string(index=False))

# Test avec une requête
compare_similarity_scores("Saveur menthe", tfidf_matrix, vectorizer, bm25, preprocessed_descriptions, df)



🔍 Requête : Saveur menthe

                               nom_produit                               saveur  tfidf_score  bm25_score
Menthe Glaciale Sel de Nicotine Alfaliquid       lime, parfait, menthol, menthe     0.264686    6.940996
              Le Thé à la Menthe 60ml Pulp           thé, lime, menthol, menthe     0.326076    6.652867
                         USA Menthe D'LICE                         lime, menthe     0.251866    6.546644
                     Menthe Fraiche D'LICE                lime, menthol, menthe     0.241657    5.983987
                 A&L Menthe Extra Glaciale lime, fruits rouges, menthol, menthe     0.224189    5.845460


In [123]:
def compare_models(query, tfidf_matrix, vectorizer, bm25, preprocessed_descriptions, df):
    print("\n--- Résultats avec TF-IDF ---")
    recommend_by_query(query, tfidf_matrix, vectorizer)  # TF-IDF

    print("\n--- Résultats avec BM25 ---")
    recommend_by_bm25(query, bm25, preprocessed_descriptions, df)  # BM25

# Test avec une requête
compare_models("Saveur menthe", tfidf_matrix, vectorizer, bm25, preprocessed_descriptions, df)




--- Résultats avec TF-IDF ---
Produits recommandés pour la requête 'Saveur menthe':

Produit 1274: Le Menthe Verte Glacée reprend lessaveurs fraîches d'une menthe chlorophylle, dans une recette signée PULP. Prêt à vaper(PAV), il vous est proposé dans plusieurs taux de nicotine au choix (0, 3, 6 et 12mg/ml), à sélectionner à la commande. Le Menthe Verte Glacée est élaboré à partir d'une base PG/VG de 70/30, et vous est livré dans une fiole de 10ml. Le Menthe Verte Glacée est un e-liquide exclusivement destiné au réservoir de votre cigarette électronique. Il est fabriqué en France par Pulp, à partir d'arômes alimentaires, de propylène glycol et de glycérine végétale. Le e-liquide Menthe Verte Glacée vous est livré dans un flacon de 10ml, équipé d'un bouchon avec sécurité enfant et d'un embout fin facilitant le remplissage de votre réservoir. Qui est PULP ? C'est dans leur laboratoire français, et plus particulièrement parisien que les aromaticiens de la marque PULP conçoivent leurs mult